#  Numba vectorize

<div class="dateauthor">
21 June 2022 | Jan H. Meinke
</div>

Numba offers a decorator `@vectorize` that allows us to generate **fast** [ufuncs](https://numpy.org/doc/stable/reference/ufuncs.html). 

In [1]:
import numba
import numpy
import numpy as np
import matplotlib.pyplot as plt

## A simple trig function

Let's implement a simple trig function:

In [2]:
import math

In [3]:
def sinacosb(a, b):
    """Calculate the product of sin(a) and cos(b)"""
    return math.sin(a) * math.cos(b)

## Passing numpy arrays as arguments

In [4]:
n = 1000000
a = np.ones(n, dtype='int8')
b = 2 * a

In [5]:
# sinacosb(a,b)

The function sinasinb is only defined for scalars, so we have to do something if we want to pass an array.

## numpy.vectorize

NumPy provides the function `vectorize`.

In [6]:
npsinacosb = np.vectorize(sinacosb)

In [ ]:
%timeit npsinacosb(a,b)

## numba.vectorize

### Dynamic ufuncs

In [8]:
usinacosb = numba.vectorize(sinacosb)

In [ ]:
%timeit usinacosb(a,b)

The function usinacosb is a *dynamic ufunc*. The arguments are determined when the function is called and only then is the function compiled.

### Eager compilation

Assume, we know with what kind of arguments a function is called, then numba can generate code as soon as we call numba vectorize. The decorator can take a list of [type specification](https://numba.readthedocs.io/en/stable/reference/types.html#signatures) strings of the form "f8(f8, f8)", where the type before the parentheses is the return type and the types within the parentheses are the argument types.

In [10]:
@numba.vectorize(['f8(i8,i8)', 'f4(f4,f4)', 'f8(f8,f8)'], nopython=True)
def usinacosb(a,b):
    return math.sin(a) * math.cos(b)

### target

If I use eager compilation I can give an addition keyword argument: *target*.

target="cpu": default, run in a single thread on the CPU

target="parallel": run in multiple threads

target="cuda": run on a CUDA-capable GPU

In [11]:
numba.set_num_threads(2)

In [12]:
pusinacosb = numba.vectorize(['f8(i8,i8)', 'f4(f4,f4)', 'f8(f8,f8)', ], nopython=True, target="parallel")(sinacosb)

In [ ]:
%timeit pusinacosb(a,b)

In [14]:
n = 100_000_000
a = np.ones(n, dtype='int8')
b = 2 * a

In [ ]:
%timeit usinacosb(a, b)
%timeit pusinacosb(a, b) 

## Exercise: The Mandelbrot set

The Mandelbrot set is the set of points *c* in the complex plane for which

$$z_{i+1} = z_i^2 + c$$

does not diverge.

The series diverges if $|z_i|>2$ for any *i*.

Since it is impracticable to calculate an infinite number of iterations, one usually sets an upper limit for the number of iterations, for example, 20.

### Escape time algorithm

A simple implementation of this algorithm is the following:

In [16]:
def escape_time(p, maxtime):
    """Perform the Mandelbrot iteration until it's clear that p diverges
    or the maximum number of iterations has been reached.
    
    Parameters
    ----------
    p: complex
        point in the complex plane
    maxtime: int
        maximum number of iterations to perform before p is considered in 
        the Mandelbrot set.
    """
    z = 0j
    for i in range(maxtime):
        z = z ** 2 + p
        if abs(z) > 2:
            return i
    return maxtime

### Todo:

1. Generate a grid of size n times m of complex numbers with the real part taken from the interval [-2.2, 1.5] and the imaginary part taken from the interval [-1.5, 1.5]. Hint, numpy.meshgrid can help.

2. Vectorize escape_time using numba.vectorize and apply it to the array above. Note, the  output is an integer.

3. Visualize the generated array using matplotlib.pyplot.imshow. 


In [17]:
# Solution 1. (c.f. Think Vector)
# Let's take the same ranges for real and imaginary numbers as before
re = numpy.linspace(-2.2, 1, 640)
im = numpy.linspace(-1.2j, 1.2j, 480)
# X and Y will have a shape of (640, 480) with the values taken from re and im.
X, Y = numpy.meshgrid(re, im)
# Finally, add X and Y to get our grid
P = X + Y

In [18]:
# Solution 2.
escape_time_vec = numba.vectorize(escape_time) 
M = escape_time_vec(P, 50)

In [ ]:
plt.imshow(M)

In [ ]:
%timeit M = escape_time_vec(P, 50)